In [3]:
!ls -la ~/Data/Instacart

total 732888
drwxr-xr-x 3 ike ike      4096 Feb 15 23:16 .
drwxr-xr-x 3 ike ike      4096 Feb 10 13:45 ..
-rw-rw-rw- 1 ike ike      2603 Jun 10  2017 aisles.csv
-rw-rw-rw- 1 ike ike       270 Jun 10  2017 departments.csv
-rw-r--r-- 1 ike ike  27681062 Feb 15 23:17 log.prior.small.csv
-rw-r--r-- 1 ike ike   1191183 Feb 15 23:17 log.train.small.csv
drwxr-xr-x 3 ike ike      4096 Feb 10 21:57 mk
-rw-rw-rw- 1 ike ike 577550706 Jun 10  2017 order_products__prior.csv
-rw-rw-rw- 1 ike ike  24680147 Jun 10  2017 order_products__train.csv
-rw-rw-rw- 1 ike ike 108968645 Jun 10  2017 orders.csv
-rw-r--r-- 1 ike ike   5238924 Feb 15 23:16 orders.small.csv
-rw-rw-rw- 1 ike ike   2166953 Jun 10  2017 products.csv
-rw-rw-rw- 1 ike ike   1475693 Jun 10  2017 sample_submission.csv
-rw-r--r-- 1 ike ike   1475693 Feb 15 23:17 sample_submission.small.csv


In [4]:
!rm ~/Data/Instacart/mk -rf

In [1]:
import pandas as pd
import numpy as np
from setting import *

orders = pd.read_csv(DATA + '/orders.small.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32}, engine='c').\
        rename(columns = {'order_dow': 'order_week', 'order_hour_of_day': 'order_hour', 'days_since_prior_order': 'order_gap'})


In [2]:
log=pd.read_csv(DATA + '/log.prior.small.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8}, usecols=['order_id', 'product_id', 'reordered'], engine='c' )
log.sort_values(by=['order_id', 'product_id'], inplace=True)
log = log.reset_index(drop=True)


In [4]:
log_extended = log.merge(orders[['order_id', 'user_id']], on='order_id', how='left')
item_user = log_extended.groupby(['product_id', 'user_id'])['order_id'].count().\
rename('item_user_cnt').to_frame().reset_index()

In [7]:
item_user['item_user_oneshot'] = (item_user.item_user_cnt==1)*1

In [10]:
item_user = item_user.groupby('product_id')['item_user_oneshot'].agg(['count', 'sum']).\
rename(columns = {'sum': 'item_oneshot_user_cnt'})


NameError: name 'item' is not defined

In [14]:
item_user['item_oneshot_user_percent'] = item_user['item_oneshot_user_cnt']/item_user['count']


ValueError: labels ['sum'] not contained in axis

In [15]:
item_user.drop(columns='count', inplace=True)

In [17]:
item_user.to_csv(DATA+'/item_oneshot_user.csv')

In [16]:
item_user.head()

,item_oneshot_user_cnt,item_oneshot_user_percent
product_id,,
1,15,0.652174
2,2,0.500000
3,2,0.500000
4,3,0.375000
7,2,1.000000
